In [16]:
import pandas as pd
import numpy as np

inputFile = 'dataset/training.seq.9'

In [17]:
df = pd.read_table(inputFile, header=None)
df.columns = ["state"]
new_df = pd.DataFrame({'state':df['state'].iloc[::2].values, 'step':df['state'].iloc[1::2].values})

# new_df['change'] = (step == #).(bool is different from previous bool) & (sep == # bool but shift down one row)
new_df['change'] = new_df.step.eq("#").diff() & new_df.step.shift().eq("#")
new_df['sol'] = new_df["change"].cumsum()

#combine state and step into one column
new_df['state'] = new_df['state'] + ',' + new_df['step']
new_df.drop(columns=['step','change'],inplace=True)

#make new dataframe with length of solution per sol
a = new_df.groupby('sol').size().sort_values(ascending=True)
a = pd.DataFrame(a)
a.columns = ["sol_len"]

#set new indices using indices of a
out = new_df.set_index('sol').loc[a.index]

#group by solution and unstck to pivot from rows to columns per step
output = out.groupby('sol',sort=False)['state'].apply(lambda out: out.reset_index(drop=True)).unstack()

In [18]:
# print(df)
# change output files to match input name
out.to_csv('dataset/training9.csv')
output.to_csv('dataset/training9_sort.csv')